In [1]:
!pwd

/Users/sanjaymahto/Chicken-Disease-Classifiaction/research


In [2]:
import os 
os.chdir('../')

In [3]:
!pwd

/Users/sanjaymahto/Chicken-Disease-Classifiaction


In [4]:
# In evaluation I don't need any configuration file 


In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

OSError: No file or directory found at artifacts/training/model.h5

In [ ]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size : list
    params_batch_size: int

In [ ]:
from src.constants import * 
from src.utils.common import read_yaml, create_directories, save_json
# Becasue I want to save those metric into json formate. 

In [ ]:
# Now it's time prepare configuration manager 
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model='artifacts/training/model.h5', 
            training_data='artifacts/data_ingestion/Chicken-fecal-images', 
            all_params=self.params,
            params_batch_size= self.params.BATCH_SIZE,
            params_image_size = self.params.IMAGE_SIZE
        )
        
        return eval_config

In [ ]:
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.2
        )
        
        dataflow_kwargs = dict(
            target_size= self.config.params_image_size[: -1], 
            batch_size = self.config.params_batch_size, 
            interpolation = 'bilinear'
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data, 
            subset='validation',
            shuffle=False, 
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        

In [ ]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config) 
    evaluation.evaluation()
    evaluation.save_score()
    
except Exception as e:
    raise e 
    

[2024-08-06 13:02:54,883: INFO: common: yaml file: /Users/sanjaymahto/Chicken-Disease-Classifiaction/config/config.yaml loaded successfully]
[2024-08-06 13:02:54,885: INFO: common: yaml file: /Users/sanjaymahto/Chicken-Disease-Classifiaction/params.yaml loaded successfully]
[2024-08-06 13:02:54,886: INFO: common: created directory at: artifacts]
Found 78 images belonging to 2 classes.
3/3 [==============================] - 8s 2s/step - loss: 4.1163 - accuracy: 0.5000
[2024-08-06 13:03:03,503: INFO: common: json file saved at: scores.json]
